## Import 

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

## Softmax

Convert numbers to probabilities with softmax

\begin{align}
P(class = i) = \frac{e^i}{ \sum{e^i}}
\end{align}

\begin{eqnarray}
P(class = 1) = \frac{e^1}{ e^1 + e^2 + e^3} \\
P(class = 2) = \frac{e^2}{ e^1 + e^2 + e^3} \\
P(class = 3) = \frac{e^3}{ e^1 + e^2 + e^3}
\end{eqnarray}

In [3]:
z = torch.FloatTensor([1, 2, 3])

PyTorch has a softmax function 

In [4]:
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


Since they are probabilities, they should add up to 1

In [5]:
hypothesis.sum()

tensor(1.)

## Cross Entropy Loss (Low-level)

For multi-class classification, we use the cross entropy loss

\begin{align}
L = \frac{1}{N} \sum{-y} \log{(y')}
\end{align}

where $\ y' $ is the predicted probability and $\ y $ is the correct probability (0 or 1)

In [6]:
z = torch.rand(3, 5, requires_grad=True) # #ofclass=5, #ofsample=3
hypothesis = F.softmax(z, dim=1)
print(z)
print(hypothesis) ## prediction

tensor([[0.7576, 0.2793, 0.4031, 0.7347, 0.0293],
        [0.7999, 0.3971, 0.7544, 0.5695, 0.4388],
        [0.6387, 0.5247, 0.6826, 0.3051, 0.4635]], requires_grad=True)
tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward>)


In [7]:
y = torch.randint(5, (3,)).long()
print(y) ## label

tensor([0, 2, 1])


In [8]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [9]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4689, grad_fn=<MeanBackward1>)


## Cross Entropy Loss with torch.nn.functional

In [10]:
# Low level
torch.log(F.softmax(z, dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward>)

In [11]:
# High level
F.log_softmax(z, dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward>)

In [12]:
# Higher level
F.nll_loss(F.log_softmax(z, dim=1), y) # Negative Log Likelihood

tensor(1.4689, grad_fn=<NllLossBackward>)

In [13]:
# The highest level
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward>)

## Training a prediction model

In [14]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]

x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [15]:
# 변수 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(3, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs+1):
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost {:.6f}'.format(
            epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost 1.098612
Epoch  100/1000 Cost 0.704199
Epoch  200/1000 Cost 0.622999
Epoch  300/1000 Cost 0.565717
Epoch  400/1000 Cost 0.515291
Epoch  500/1000 Cost 0.467662
Epoch  600/1000 Cost 0.421278
Epoch  700/1000 Cost 0.375401
Epoch  800/1000 Cost 0.329766
Epoch  900/1000 Cost 0.285072
Epoch 1000/1000 Cost 0.248155


## High-level Implementation with nn.Module

In [16]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3)
        
    def forward(self, x):
        return self.linear(x)
    
model = SoftmaxClassifierModel()

In [17]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    prediction = model(x_train)
    cost = F.cross_entropy(prediction, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost {:.6f}'.format(
            epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost 1.849513
Epoch  100/1000 Cost 0.689894
Epoch  200/1000 Cost 0.609259
Epoch  300/1000 Cost 0.551218
Epoch  400/1000 Cost 0.500141
Epoch  500/1000 Cost 0.451947
Epoch  600/1000 Cost 0.405051
Epoch  700/1000 Cost 0.358733
Epoch  800/1000 Cost 0.312912
Epoch  900/1000 Cost 0.269521
Epoch 1000/1000 Cost 0.241922
